In [ ]:
import os
import kagglehub

path = kagglehub.dataset_download("noulam/tomato")
path = path + '/New Plant Diseases Dataset(Augmented)'
print("Dataset path:",path)

print("Files:", os.listdir(path))


Dataset path: /root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)
Files: ['valid', 'train']


In [ ]:
!nvidia-smi

Sat Mar 22 04:52:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [ ]:
# Importing the bibraries

from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [35]:
# resize all teh images to this

IMAGE_SIZE = [ 224,224]

train_path = '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train'
test_path = '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid'

In [36]:
#import the InveptionV3 library as shown below adn add preprocessing layer to hte front of InceptionV#
#Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE +[3],weights = 'imagenet', include_top=False)

In [37]:
# dont't train existing weights
for layer in inception.layers:
  layer.trainable = False

In [38]:
#useful for getting number of output classes

folders = glob('/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/*')

folders

['/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___Septoria_leaf_spot',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___Late_blight',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___Target_Spot',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___Bacterial_spot',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_mosaic_virus',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train/Tomato___healthy',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(

In [39]:
#useful for getting number of output classes

test_folders = glob('/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/*')

test_folders

['/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Septoria_leaf_spot',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Late_blight',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Target_Spot',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Bacterial_spot',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Tomato_mosaic_virus',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid/Tomato___healthy',
 '/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(

In [40]:
# Our layer - You can add more if you want

x = Flatten()(inception.output)

In [41]:
prediction = Dense(len(folders),activation='softmax')(x)

#create a model object

model = Model(inputs=inception.input,outputs= prediction)

In [42]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_94 (Conv2D)        │ (None, 111, 111, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_94    │ (None, 111, 111, 32)   │             96 │ conv2d_94[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_94             │ (None, 111, 111, 32)   │              0 │ batch_normalization_9… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_95 (Conv2D)        │ (None, 109, 109, 32)   │          9,216 │ activation_94[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_95    │ (None, 109, 109, 32)   │             96 │ conv2d_95[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_95             │ (None, 109, 109, 32)   │              0 │ batch_normalization_9… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_96 (Conv2D)        │ (None, 109, 109, 64)   │         18,432 │ activation_95[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_96    │ (None, 109, 109, 64)   │            192 │ conv2d_96[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_96             │ (None, 109, 109, 64)   │              0 │ batch_normalization_9… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 54, 54, 64)     │              0 │ activation_96[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_97 (Conv2D)        │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_97    │ (None, 54, 54, 80)     │            240 │ conv2d_97[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_97             │ (None, 54, 54, 80)     │              0 │ batch_normalization_9… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_98 (Conv2D)   

 Total params: 22,314,794 (85.12 MB)

 Trainable params: 512,010 (1.95 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [43]:
# tell the model what cost and optimization method to use

model.compile(
    loss = 'categorical_crossentropy',
    optimizer= 'adam',
    metrics = ['accuracy']
)

In [44]:
# Use the Image Data Generator to improt the images from teh dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [45]:
#Make sure you provie the same target size as initalized for the image size

traning_set = train_datagen.flow_from_directory('/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                class_mode = 'categorical')

Found 18345 images belonging to 10 classes.


In [46]:
#Make sure you provie the same target size as initalized for the image size

testing_set = test_datagen.flow_from_directory('/root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)/valid',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [49]:
# fint eh model
# runt he cell it will take some time to execute

r = model.fit(
    traning_set,
    validation_data = testing_set,
    epochs = 15,
    steps_per_epoch = len(traning_set),
    validation_steps = len(testing_set)
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 256s 418ms/step - accuracy: 0.5979 - loss: 4.2056 - val_accuracy: 0.7627 - val_loss: 2.2834
Epoch 2/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 223s 388ms/step - accuracy: 0.7703 - loss: 2.4768 - val_accuracy: 0.8242 - val_loss: 1.8193
Epoch 3/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 265s 394ms/step - accuracy: 0.8318 - loss: 1.8606 - val_accuracy: 0.7939 - val_loss: 3.2055
Epoch 4/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 225s 392ms/step - accuracy: 0.8345 - loss: 2.0436 - val_accuracy: 0.8362 - val_loss: 2.3232
Epoch 5/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 221s 386ms/step - accuracy: 0.8591 - loss: 1.7640 - val_accuracy: 0.8449 - val_loss: 2.4286
Epoch 6/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 224s 390ms/step - accuracy: 0.8683 - loss: 1.8283 - val_accuracy: 0.8185 - val_loss: 2.6632
Epoch 7/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 262s 391ms/step - accuracy: 0.8725 - loss: 1.6798 - val_accuracy: 0.8467 - val_loss: 2.1365
Epoch 8/15
574/574 ━━━━━━━━━━━━━━━━━━━━ 256s 381ms/step - accuracy: 0.8826 -